## 硬注意力的形式化描述

在 [Show, Attend and Tell: Neural Image Caption Generation with Visual Attention](https://arxiv.org/pdf/1502.03044v3.pdf) 中，使用了编码器-解码器架构：首先用卷积神经网络从输入图像中抽取特征得到编码，再通过循环神经网络将编码的特征解码为输出序列。

![](https://ai-studio-static-online.cdn.bcebos.com/e5c1ce881e7c4f86b55e04a5c11c554391b0de37aa9349ed8812038f2ed3e6df)

其中，通过 LSTM 解码时的注意力机制作用在输入图像的 feature map 上，文中采用了软注意力（attention）和硬注意力（hard attention）两种注意力机制。

 

模型的输入是一张 $224 \times 224$ 的图像，得到的 feature map 的维度是 $14 \times 14 \times 512$，记为 annotation vectors：
$$
\boldsymbol{a} = \left\{ \boldsymbol{a}_1,\boldsymbol{a}_2, \cdots,
\boldsymbol{a}_L \right\}, \quad \boldsymbol{a}_i \in \mathbb{R}^D
$$
其中，$L = 14 \times 14=196$ 为 feature map 的像素点个数，$D=512$ 为特征的维度。

模型生成的文本序列是与输入图片相对应的描述：
$$
\boldsymbol{y} = \left\{ \boldsymbol{y}_1, \cdots, \boldsymbol{y}_C \right\}, \quad \boldsymbol{y}_i \in \mathbb{R}^K
$$
其中，$K$ 为词表大小，$C$ 为描述文本的长度。



模型采用的解码器是 LSTM，在解码过程中第 $t$ 个时间步通过注意力机制从输入图片中相关部分得到的动态表示记为上下文向量（context vector）$\hat{\boldsymbol{z}}_t \in \mathbb{R}^D$。

在第 $t$ 个时间步的解码过程中，注意力分数由 $\boldsymbol{a}_i$ 和 $\boldsymbol{h}_{t-1}$ 得到（$i=1, \cdots, L$），之后归一化得到注意力权重 $\alpha_{t,i}$，再用这两者得到上下文向量 $\hat{\boldsymbol{z}}_t$。
$$
score_{t,i} = f_{att}\left( \boldsymbol{a}_i,\boldsymbol{h}_{t-1} \right)\\
\alpha_{t,i} = \frac{\exp \left( score_{t,i} \right)}{
\sum_{k=1}^L \exp \left( score_{t,k} \right)} \\
\hat{\boldsymbol{z}}_t = \phi \left( \left\{ \boldsymbol{a}_i \right\},\left\{ \alpha_{i} \right\}
\right)
$$



使用软注意力的计算方法直接通过对 feature map 上的每个像素点进行加权求和得到 $\hat{\boldsymbol{z}}_t$，即
$$
\mathbb{E} \left[ \boldsymbol{z}_t\right] = \sum_{i=1}^{L}
\alpha_{t,i} \boldsymbol{a}_i
$$

而硬注意力期望在当前时间步解码时，只在 feature map 上的一点给予响应，在第 $t$ 个时间步且在 feature map 上的位置 $i$ 处产生响应时对应的独热编码为 $s_{t,i}$，此时计算方式如下：
$$
p \left( s_{t,i}=1 \vert s_{j<t}, \boldsymbol{a} \right) = \alpha_{t,i} \\
\hat{\boldsymbol{z}}_t = \sum_{i=1}^L s_{t,i} \boldsymbol{a}_i
$$